In [1]:
import os
import json
from PIL import Image
from tqdm import tqdm
from collections import Counter

def convert_yolo_to_coco(image_dir, label_dir, output_json_path, class_names):
    image_id = 0
    ann_id = 0

    # Force category IDs to match class_names order (0 to N-1)
    categories = [{"id": idx, "name": name, "supercategory": "none"} for idx, name in enumerate(class_names)]
    coco_output = {
        "images": [],
        "annotations": [],
        "categories": categories
    }

    class_counter = Counter()

    image_files = sorted([f for f in os.listdir(image_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])

    for img_file in tqdm(image_files, desc=f"Converting {os.path.basename(output_json_path)}"):
        img_path = os.path.join(image_dir, img_file)
        label_path = os.path.join(label_dir, os.path.splitext(img_file)[0] + '.txt')

        try:
            with Image.open(img_path) as img:
                width, height = img.size
        except:
            print(f"[Warning] Skipping unreadable image: {img_path}")
            continue

        coco_output["images"].append({
            "id": image_id,
            "file_name": img_file,
            "width": width,
            "height": height
        })

        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) != 5:
                        print(f"[Warning] Malformed line in {label_path}: {line.strip()}")
                        continue

                    try:
                        class_id, cx, cy, w, h = map(float, parts)
                        class_id = int(class_id)
                        if not (0 <= class_id < len(class_names)):
                            print(f"[Warning] Invalid class ID {class_id} in {label_path}")
                            continue
                    except:
                        print(f"[Warning] Error parsing line in {label_path}: {line.strip()}")
                        continue

                    x = (cx - w / 2) * width
                    y = (cy - h / 2) * height
                    bw = w * width
                    bh = h * height

                    coco_output["annotations"].append({
                        "id": ann_id,
                        "image_id": image_id,
                        "category_id": class_id,
                        "bbox": [x, y, bw, bh],
                        "area": bw * bh,
                        "iscrowd": 0,
                        "segmentation": []
                    })
                    ann_id += 1
                    class_counter[class_id] += 1

        image_id += 1

    # Save to file
    with open(output_json_path, 'w') as f:
        json.dump(coco_output, f, indent=4)

    print(f"\n✅ COCO annotation saved to: {output_json_path}")
    print("📊 Class instance distribution:")
    for class_id, count in sorted(class_counter.items()):
        print(f"  {class_id} ({class_names[class_id]}): {count} objects")

# ======= Usage =======
if __name__ == "__main__":
    base_dir = "C:/KD-YOLOX-ViT/datasets"
    class_names = ['epith', 'rbc/wbc']

    for split in ['train', 'val', 'test']:
        img_dir = os.path.join(base_dir, split, 'images')
        lbl_dir = os.path.join(base_dir, split, 'labels')
        out_json = os.path.join(base_dir, 'annotations', f"instances_{split}.json")

        os.makedirs(os.path.dirname(out_json), exist_ok=True)

        convert_yolo_to_coco(img_dir, lbl_dir, out_json, class_names)


Converting instances_train.json: 100%|██████████| 60/60 [00:00<00:00, 170.51it/s]



✅ COCO annotation saved to: C:/KD-YOLOX-ViT/datasets\annotations\instances_train.json
📊 Class instance distribution:
  0 (epith): 100 objects
  1 (rbc/wbc): 301 objects


Converting instances_val.json: 100%|██████████| 30/30 [00:00<00:00, 174.77it/s]



✅ COCO annotation saved to: C:/KD-YOLOX-ViT/datasets\annotations\instances_val.json
📊 Class instance distribution:
  0 (epith): 58 objects
  1 (rbc/wbc): 251 objects


Converting instances_test.json: 100%|██████████| 210/210 [00:01<00:00, 189.12it/s]


✅ COCO annotation saved to: C:/KD-YOLOX-ViT/datasets\annotations\instances_test.json
📊 Class instance distribution:
  0 (epith): 412 objects
  1 (rbc/wbc): 2038 objects


In [ ]:
yolo_to_coco(
    yolo_dir="path/to/train/labels",
    images_dir="path/to/train/images",
    output_json="path/to/annotations/instances_train.json",
    class_names=class_names
)

yolo_to_coco(
    yolo_dir="path/to/val/labels",
    images_dir="path/to/val/images",
    output_json="path/to/annotations/instances_val.json",
    class_names=class_names
)

yolo_to_coco(
    yolo_dir="path/to/test/labels",
    images_dir="path/to/test/images",
    output_json="path/to/annotations/instances_test.json",
    class_names=class_names
)
